<a href="https://colab.research.google.com/github/skyprince999/Data-Engineering-Covid19-ETL/blob/master/AWS_ES_Snapshot_Restore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!curl ipecho.net/plain # Get ip of Colab notebook # 35.204.243.12 #35.196.39.8

In [ ]:
#Install required packages
!pip3 install -U elasticsearch
!pip3 install -U jsonlines
!pip3 install requests-aws4auth

In [ ]:

# Install AWS CLIv2 
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install

In [ ]:
!aws configure   # For personal account - I use the elastic user ID 

In [ ]:

from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time
import jsonlines

import boto3
import requests
from requests_aws4auth import AWS4Auth
import pprint

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
host = 'https://search-covidtweets-XXXXXXXXXXXXXXXXXXXXXXXXXXXXX.es.amazonaws.com/' #domain host ip for accessing the elasticsearch domain
region = 'us-east-1' # For example, us-west-1
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
role = 'arn:aws:iam::XXXXXXXXXXXXXXXXXXXX:role/service-role/elasticLambdaRole' # role for accessing ES and lambda function

In [ ]:
# Register repository
path = '_snapshot/es-covid-backup' # the Elasticsearch API endpoint
url = host + path

payload = {
  "type": "s3",
  "settings": {
    "bucket": "es-covid-backup",
    "region": region,
    "role_arn": role
  }
}

headers = {"Content-Type": "application/json"}

r = requests.put(url, auth=awsauth, json=payload, headers=headers)

print(r.status_code)
print(r.text)

200
{"acknowledged":true}


In [ ]:
  # Once S3 repository is registered, we can get details of the available snapshot
path = '_snapshot/_all' # the Elasticsearch API endpoint
url = host + path

payload = {
  "type": "s3",
  "settings": {
    "bucket": "es-covid-backup",
    "region": region,
    "role_arn": role
  }
}

headers = {"Content-Type": "application/json"}

r = requests.get(url, auth=awsauth, headers=headers)

print(r.status_code)
print(r.text)

In [ ]:
# Get list of available snapshot
path = '_cat/snapshots/es-covid-backup'
url = host + path

r = requests.get(url, auth=awsauth)

print(r.text)


covidtweets-14082020    SUCCESS 1597421569 16:12:49 1597422119 16:21:59  9.1m 2 3 0 3
covidtweets-14082020_v2 SUCCESS 1597425559 17:19:19 1597425582 17:19:42 22.6s 2 3 0 3



In [ ]:
# Take a snapshot
path = '_snapshot/es-covid-backup/covidtweets-14082020_v3'
url = host + path

r = requests.put(url, auth=awsauth)

print(r.text)


{"accepted":true}


In [ ]:
# check status of  snapshot
path = '_snapshot/es-covid-backup/covidtweets-14082020_v2/_status'
url = host + path

r = requests.get(url, auth=awsauth)

print(r.text)


In [ ]:
# # Delete index

# path = 'covidtweets'#'covidtweets' # '.kibana_1'#
# url = host + path

# r = requests.delete(url, auth=awsauth)

# print(r.text)

In [ ]:
# Restore snapshot (all indices except Kibana and fine-grained access control)

path = '_snapshot/es-covid-backup/covidtweets-14082020_v2/_restore'
url = host + path

# # Restore all indices
# # payload = {
# #   "indices": "-.kibana*,-.opendistro_security",
# #   "include_global_state": False
# # }


# # Restore snapshot (one index)

payload = {"indices": "covidtweets"}

headers = {"Content-Type": "application/json"}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)

print(r.text)


{"accepted":true}


In [ ]:
path = 'covidtweets/_recovery' # Get status on index recovery
url = host + path
r = requests.get(url, auth=awsauth, headers=headers)

print(r.text)
